In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24260565
paper_name = 'gaytan_vulpe_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [29]:
original_data = pd.read_excel('raw_data/journal.pone.0081253.s003.xlsx', sheet_name='mutants with altered growth in ', skiprows=5)

In [30]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 677 x 8


In [31]:
original_data.head()

,ORF,25% IC20 (160 μM),50% IC20 (320 μM),100% IC20 (640 μM),Name,Description of deleted gene,Unnamed: 6,Web.based.Db
0,YOR153W,-6.30,-5.80,-6.05,PDR5,Plasma membrane ATP-binding cassette (ABC) tra...,NaN,http://db.yeastgenome.org/cgi-bin/locus.pl?loc...
1,YDL020C,-3.15,-3.05,-2.55,RPN4,Transcription factor that stimulates expressio...,NaN,http://db.yeastgenome.org/cgi-bin/locus.pl?loc...
2,YIR023W,-3.10,-3.50,-3.00,DAL81,Positive regulator of genes in multiple nitrog...,NaN,http://db.yeastgenome.org/cgi-bin/locus.pl?loc...
3,YDR354W,-2.90,-4.50,-4.70,TRP4,Anthranilate phosphoribosyl transferase of the...,NaN,http://db.yeastgenome.org/cgi-bin/locus.pl?loc...
4,YDR126W,-2.90,-3.20,-3.00,SWF1,Palmitoyltransferase that acts on the SNAREs S...,NaN,http://db.yeastgenome.org/cgi-bin/locus.pl?loc...


In [32]:
original_data['orf'] = original_data['ORF'].astype(str)

In [33]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [34]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [35]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, 25% IC20 (160 μM), 50% IC20 (320 μM), 100% IC20 (640 μM), Name, Description of deleted gene, Unnamed: 6, Web.based.Db, orf]
Index: []


In [36]:
original_data.set_index('orf', inplace=True)

In [37]:
original_data = original_data[['25% IC20 (160 μM)','50% IC20 (320 μM)','100% IC20 (640 μM)']].copy()

In [38]:
original_data[original_data.isnull()] = 0

In [39]:
original_data = original_data.groupby(original_data.index).mean()

In [40]:
original_data.shape

(677, 3)

# Prepare the final dataset

In [41]:
data = original_data.copy()

In [42]:
dataset_ids = [205, 204, 203]
datasets = datasets.reindex(index=dataset_ids)

In [43]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [44]:
data.head()

dataset_id,205,204,203
data_type,value,value,value
orf,,,
YAL002W,1.3,1.0,0.0
YAL053W,-0.9,-1.1,0.0
YAL064C-A,1.3,1.1,0.0
YAR031W,0.0,-0.8,-0.9
YBL007C,0.0,0.0,1.5


## Subset to the genes currently in SGD

In [45]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [46]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,205,204,203
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,1.3,1.0,0.0
49,YAL053W,-0.9,-1.1,0.0
1867,YAL064C-A,1.3,1.1,0.0
75,YAR031W,0.0,-0.8,-0.9
95,YBL007C,0.0,0.0,1.5


# Normalize

In [47]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [48]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [49]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,205,204,203,205,204,203
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,1.3,1.0,0.0,1.995047,1.493957,-0.004421
49,YAL053W,-0.9,-1.1,0.0,-1.389075,-1.668911,-0.004421
1867,YAL064C-A,1.3,1.1,0.0,1.995047,1.644569,-0.004421
75,YAR031W,0.0,-0.8,-0.9,-0.004661,-1.217073,-1.317388
95,YBL007C,0.0,0.0,1.5,-0.004661,-0.012171,2.183859


# Print out

In [50]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [51]:
from IO.save_data_to_db3 import *

In [52]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 24260565...
Inserting the new data...


100%|██████████| 3/3 [00:03<00:00,  1.08s/it]

Updating the data_modified_on field...
